In [1]:
%pip install pandas
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Ejercicio 3: Conteo palabra en campo 'jobtile'

In [11]:
import pyspark
from pyspark.sql import SparkSession
from pymongo import MongoClient
import collections
import pandas as pd

# Crea una SparkSession
spark = SparkSession.builder \
    .appName("MongoSparkConnector2") \
    .getOrCreate()

# Conecta a MongoDB usando pymongo
client = MongoClient("mongodb://172.17.0.3:27017/")
db = client['docker']
collection = db['perfilesmongo']

# Extrae los datos desde MongoDB
mongo_data = list(collection.find())

#print(mongo_data)

# Convierte los datos a un DataFrame de pandas
pdf = pd.DataFrame(mongo_data)

# Elimina la columna '_id' si es necesario, ya que no es serializable por defecto en Spark
if '_id' in pdf.columns:
    pdf = pdf.drop(columns=['_id'])

# Convierte el DataFrame de pandas a un DataFrame de Spark
df = spark.createDataFrame(pdf)

# Convertir el DataFrame a un RDD
rdd = df.rdd
#df.show()

rdd = df.select("location").rdd
    
rdd_filtrado = rdd.map(lambda x: (x['location']))
wordsCounts = rdd_filtrado.map(lambda x: (x, 1)).reduceByKey(lambda x , y: x + y)

wordsCountsSorted = wordsCounts.map(lambda x: (x[1], x[0])).sortByKey()

results = wordsCountsSorted.collect()

for result in results:
    print(result[0])
    print(result[1])

1
Torrelodones
1
Community of Madrid, Spain
1
L'Hospitalet de Llobregat
1
Majadahonda
1
Greater Salamanca Metropolitan Area
1
Cervelló
1
Aranjuez
1
Alcobendas
1
Coslada
1
Greater Sevilla Metropolitan Area
1
Ourense
1
Pinto
1
Badalona
1
Greater Bilbao Metropolitan Area
1
Greater Granada Metropolitan Area
1
United States
1
Segovia
1
Godella
1
Mollet del Vallès
1
Balaguer
1
Collado-Villalba
1
Calella
1
Argentona
1
Loeches
1
Greater Las Palmas Metropolitan Area
1
Inca
1
Greater Málaga Metropolitan Area
1
Tres Cantos
1
Parla
1
Málaga
1
Mejorada del Campo
1
Calafell
2
Leganés
2
Alcalá de Henares
2
Móstoles
2
Granada
2
San Fernando de Henares
3
Fuenlabrada
3
Torrejón de Ardoz
4
San Sebastián de los Reyes
4
Greater Barcelona Metropolitan Area
7
Getafe
11
Spain
18
Barcelona
110
Greater Madrid Metropolitan Area
184
Madrid
